In [10]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


In [11]:
!C:\ProgramData\anaconda3\python.exe -m pip install librosa==0.10.0.post2 soundfile

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import librosa
print("Librosa version:", librosa.__version__)


Librosa version: 0.10.0.post2


In [13]:
import os

# Path to your RAVDESS dataset inside your project
DATASET_PATH = "../data/Audio_Speech_Actors_01-24/"


# Emotion map from RAVDESS codes
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# Collect file paths and labels
audio_files = []
labels = []

for actor in os.listdir(DATASET_PATH):
    actor_path = os.path.join(DATASET_PATH, actor)
    
    if not os.path.isdir(actor_path):
        continue

    for file in os.listdir(actor_path):
        if file.endswith(".wav"):
            file_path = os.path.join(actor_path, file)
            emotion_code = file.split("-")[2]
            label = emotion_map.get(emotion_code)

            audio_files.append(file_path)
            labels.append(label)

print(f"Total audio files: {len(audio_files)}")
print(f"First 5 labels: {labels[:5]}")
print(f"First 5 files:\n{audio_files[:5]}")


Total audio files: 1440
First 5 labels: ['neutral', 'neutral', 'neutral', 'neutral', 'calm']
First 5 files:
['../data/Audio_Speech_Actors_01-24/Actor_01\\03-01-01-01-01-01-01.wav', '../data/Audio_Speech_Actors_01-24/Actor_01\\03-01-01-01-01-02-01.wav', '../data/Audio_Speech_Actors_01-24/Actor_01\\03-01-01-01-02-01-01.wav', '../data/Audio_Speech_Actors_01-24/Actor_01\\03-01-01-01-02-02-01.wav', '../data/Audio_Speech_Actors_01-24/Actor_01\\03-01-02-01-01-01-01.wav']


In [14]:
from collections import Counter

emotion_counts = Counter(labels)
print("Emotion counts:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count}")


Emotion counts:
neutral: 96
calm: 192
happy: 192
sad: 192
angry: 192
fearful: 192
disgust: 192
surprised: 192


In [23]:
from IPython.display import Audio

print("Example file:", audio_files[4])
print("Label:", labels[4])
Audio(filename=audio_files[4])

Example file: ../data/Audio_Speech_Actors_01-24/Actor_01\03-01-02-01-01-01-01.wav
Label: calm


In [16]:
import numpy as np
import librosa

def extract_mfcc(file_path, n_mfcc=40, max_len=173):
    # Load audio
    y, sr = librosa.load(file_path, sr=None)

    # Compute MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T  # (time, features)

    # Make all samples the same length
    if mfcc.shape[0] < max_len:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode="constant")
    else:
        mfcc = mfcc[:max_len, :]

    return mfcc


In [17]:
sample_mfcc = extract_mfcc(audio_files[0])
print("MFCC shape:", sample_mfcc.shape)
print("Label for this file:", labels[0])


MFCC shape: (173, 40)
Label for this file: neutral


In [18]:
import numpy as np

# Unique emotions
unique_emotions = sorted(set(labels))
print("Emotions:", unique_emotions)

# Map emotion -> index
label_to_idx = {emo: i for i, emo in enumerate(unique_emotions)}
idx_to_label = {i: emo for emo, i in label_to_idx.items()}

print("Label to index:", label_to_idx)


Emotions: ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Label to index: {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}


In [29]:
X = []
y_idx = []

for path, lab in zip(audio_files, labels):
    mfcc = extract_mfcc(path)          # (173, 40)
    X.append(mfcc)
    y_idx.append(label_to_idx[lab])    # رقم الليبل

X = np.array(X, dtype=np.float32)      # (N, 173, 40)
y = np.array(y_idx, dtype=np.int64)    # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1440, 173, 40)
y shape: (1440,)


In [30]:
X = X[..., np.newaxis]   # (N, 173, 40, 1)
print("X shape after adding channel:", X.shape)

X shape after adding channel: (1440, 173, 40, 1)


In [31]:
!C:\ProgramData\anaconda3\python.exe -m pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [32]:
from sklearn.model_selection import train_test_split

# 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape,   y_val.shape)
print("Test: ", X_test.shape,  y_test.shape)


Train: (1008, 173, 40, 1) (1008,)
Val:   (216, 173, 40, 1) (216,)
Test:  (216, 173, 40, 1) (216,)
